In [1]:
import pandas as pd
import requests

def get_headers():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/90.0.4430.212 Safari/537.36'
    }
    return headers

class get_11v11:
    def __init__(self, date, venue = None):
        self.date = date
        self.venue = venue
        
        self.url = self.get_url()

        r = requests.get(self.url, headers = get_headers())

        self.tables = pd.read_html(r.content, flavor="bs4")
        
        self.table = self.get_table()
        self.pos = self.get_pos()

    def get_url(self):
        year = self.date[:4]
        month = pd.to_datetime(self.date).month_name().lower()
        day = self.date[8:]

        url = f"https://www.11v11.com/league-tables/league-two/{day}-{month}-{year}/"
        if self.venue and self.venue[0].upper() in ["H", "A"]:
            if self.venue[0].upper() == "H":
                self.venue = "home"
            else:
                self.venue = "away"
            url += f"{self.venue}"
        return url
    
    def get_table(self):
        table = self.tables[0]
        table.Pos = table.Pos.index + 1
        return table
    
    def get_pos(self):
        try:
            pos = self.table.query("Team.str.contains('Tranmere')").Pos.values[0]
        except:
            pos = "No table containing Tranmere Rovers found"
        return pos

In [2]:
class bbc_api:
    def __init__(self, date):
        self.date = date

        self.match_url = self.get_match_url()
        self.match_list = self.get_match_list()
        try:
            self.match_key = self.get_match_key()
            self.event_key = self.get_event_key()
            self.tournament_data = self.get_tournament_data()
            self.match_data = self.get_match_data()
            self.tranmere, self.opponent = self.get_teams()
            self.teams = self.print_teams()

            self.lineup_url = self.get_lineup_url()
            self.lineup_data = self.get_lineup_data()
            self.tranmere_players = self.lineup_data["teams"][self.tranmere]["players"]
            self.opp_players = self.lineup_data["teams"][self.opponent]["players"]
            self.attendance = self.get_attendance()
            self.referee = self.get_referee()
        except Exception as e:
            return None

    def get_match_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-football-scores-match-list-data/endDate/{self.date}/startDate/{self.date}/team/tranmere-rovers/todayDate/{self.date}/version/2.4.6/withPlayerActions/true?timeout=5"
        return url
    
    def get_match_list(self):
        r = requests.get(self.match_url, headers = get_headers())
        match_list = r.json()

        if not match_list['matchData']:
            print(f"No matches found for {self.date}")
            return 
        else:
            return match_list
    
    def get_match_key(self):
        match_key = next(iter(self.match_list["matchData"][0]["tournamentDatesWithEvents"]))
        return match_key
    
    def get_event_key(self):
        event_key = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]["eventKey"]
        return event_key

    def get_tournament_data(self):
        tournament_data = self.match_list["matchData"][0]["tournamentMeta"]
        return tournament_data    
    
    def get_match_data(self):
        match_data = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]
        return match_data
        
    def get_lineup_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-sport-football-team-lineups-data/event/{self.event_key}/version/1.0.8"
        return url
    
    def get_lineup_data(self):
        r = requests.get(self.lineup_url, headers = get_headers())
        lineup_data = r.json()
        return lineup_data
    
    def get_teams(self):
        if self.match_data["homeTeam"]["name"]["full"] == "Tranmere Rovers":
            tranmere = "homeTeam"
            opponent = "awayTeam"
        else:
            tranmere = "awayTeam"
            opponent = "homeTeam"
        return tranmere, opponent
    
    def print_teams(self):
        second_team = self.match_data[self.opponent]["name"]["full"]
        print(f"Tranmere are {self.tranmere}. {second_team} are {self.opponent}")
             
    def get_attendance(self):
        if "attendance" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["attendance"].replace(",", "")
            return attendance
        else:
            print(f"No attendance data for {self.date}")
            return None
        
    def get_referee(self):
        if "referee" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["referee"].replace(",", "")
            return attendance
        else:
            print(f"No referee for {self.date}")
            return None


In [ ]:
date = "2023-08-05"

data = bbc_api(date)

match_data = bbc_api(date).match_data

In [19]:
class match_df:
    def __init__(self, date):
        self.date = date
        self.data = bbc_api(self.date)
        self.match_data = self.data.match_data
        self.players = self.data.tranmere_players

        self.goals = self.get_goals_df()
        self.player_apps = self.get_apps()
        self.cards = self.get_cards()
        self.yellow_cards = self.get_yellow_cards()
        self.red_cards = self.get_red_cards()

    def get_goals_df(self):
        player_actions = self.match_data[self.data.tranmere]["playerActions"]

        goals = []
        for player in player_actions:
            player_name = player["name"]["full"]    
            actions = player["actions"]
            for action in actions:
                if action["type"] == "goal":
                    goal = {
                        "game_date": self.date,
                        "player_name": player_name,
                        "goal_min": action["timeElapsed"],
                        "penalty": action["penalty"],
                        "own_goal": action["ownGoal"]
                    }
                    goals.append(goal)
            
        goals_df = pd.DataFrame(goals)
        return goals_df
    
    def get_apps(self):
        player_apps = []
        for player in self.players:
            player_name = player["name"]["full"]
            try:
                shirt_no = player["meta"]["uniformNumber"]
            except:
                shirt_no = None
            role = player["meta"]["status"].replace("bench", "sub")
            
            player_app = {
                "game_date": self.date,
                "player_name": player_name,
                "shirt_no": shirt_no,
                "role": role
            }
            player_apps.append(player_app)
        
        apps_df = pd.DataFrame(player_apps)
        return apps_df
    
    def get_cards(self):
        player_cards = []

        for player in self.players:
            player_name = player["name"]["full"]
            cards = player["bookings"]
            if cards:
                for card in cards:
                    player_card = {
                        "game_date": self.date,
                        "player_name": player_name,
                        "minute": card["timeElapsed"],
                        "card_type": card["type"]
                    }
                    player_cards.append(player_card)
        
        cards_df = pd.DataFrame(player_cards)
        return cards_df
    
    def get_yellow_cards(self):
        cards = self.cards
        yellows = cards.query("card_type == 'yellow-card'").rename(columns = {"minute": "min_yc"}).drop(columns = ["card_type"])
        if yellows.empty:
            print("No yellow cards")
            return None
        else:
            return yellows
    
    def get_red_cards(self):
        cards = self.cards
        reds = cards.query("card_type.str.contains('red')").rename(columns = {"minute": "min_so"}).drop(columns = ["card_type"])
        if reds.empty:
            print("No reds cards")
            return None
        else:
            return reds
        
match_df("2023-08-08").yellow_cards

Tranmere are awayTeam. Barnsley are homeTeam
No reds cards


,game_date,player_name,min_yc
0,2023-08-08,Jake Leake,69
1,2023-08-08,Paul Lewis,90


In [ ]:
# player_actions = match_data[data.tranmere]["playerActions"]

# goals = []
# for player in player_actions:
#     player_name = player["name"]["full"]    
#     actions = player["actions"]
#     for action in actions:
#         if action["type"] == "goal":
#             goal = {
#                 "game_date": date,
#                 "player_name": player_name,
#                 "goal_min": action["timeElapsed"],
#                 "penalty": action["penalty"],
#                 "own_goal": action["ownGoal"]
#             }
#             goals.append(goal)

# goals_df = pd.DataFrame(goals)
# if not goals_df.empty:
#     goals_df.to_csv("data-updates/goals.csv", index = False)

# goals_df

In [ ]:
# players = data.tranmere_players

# player_apps = []
# player_cards = []
player_sub_mins = []
player_subs = []

for player in players:
        player_name = player["name"]["full"]
        try:
            shirt_no = player["meta"]["uniformNumber"]
        except:
            shirt_no = None
        role = player["meta"]["status"].replace("bench", "sub")
        
        # player_app = {
        #     "game_date": date,
        #     "player_name": player_name,
        #     "shirt_no": shirt_no,
        #     "role": role
        # }
        # player_apps.append(player_app)

        # cards = player["bookings"]
        # if cards:
        #     for card in cards:
        #         player_card = {
        #             "game_date": date,
        #             "player_name": player_name,
        #             "minute": card["timeElapsed"],
        #             "card_type": card["type"]
        #         }
        #         player_cards.append(player_card)

        subs = player["substitutions"]
        if subs:
            sub_min = subs[0]["timeElapsed"]
            try:
                sub_on_no = subs[0]["replacedBy"]["meta"]["uniformNumber"]
            except:
                sub_on_no = None
            player_on = subs[0]["replacedBy"]["name"]["full"]

            sub_mins_off = {
                "game_date": date,
                "player_name": player_name,
                "min_off": sub_min,
                "min_on": None
            }
            sub_mins_on = {
                "game_date": date,
                "player_name": player_on,
                "min_off": None,
                "min_on": sub_min
            }
            player_sub_mins.append(sub_mins_off)
            player_sub_mins.append(sub_mins_on)

            sub_on = {
                "game_date": date,
                "shirt_no": sub_on_no,
                "player_name": player_name,
                "on_for": shirt_no,
                "off_for": None
            }
            sub_off = {
                "game_date": date,
                "shirt_no": shirt_no,
                "player_name": player_on,
                "on_for": None,
                "off_for": sub_on_no
            }
            player_subs.append(sub_on)
            player_subs.append(sub_off)

# new_apps = pd.DataFrame(player_apps)
# new_subs = pd.DataFrame(player_subs)
new_sub_mins = pd.DataFrame(player_sub_mins)
new_cards = pd.DataFrame(player_cards)

new_yellows = new_cards[new_cards["card_type"] == "yellow-card"].rename(columns = {"minute": "min_yc"}).drop(columns = ["card_type"])
new_reds = new_cards[new_cards["card_type"].str.contains("red")].rename(columns = {"minute": "min_so"}).drop(columns = ["card_type"])

new_apps.to_csv("data-updates/player_apps.csv", index = False)
if not new_subs.empty:
    new_subs.to_csv("data-updates/subs.csv", index = False)
    new_sub_mins.to_csv("data-updates/sub_mins.csv", index = False)
if not new_cards.empty:
    new_yellows.to_csv("data-updates/yellow_cards.csv", index = False)
    new_reds.to_csv("data-updates/red_cards.csv", index = False)

In [ ]:
# CURRENTLY UNUSED

formation = data.lineup_data["teams"][data.tranmere]["formation"]
formation = "-".join(str(formation))

stadium = match_data["venue"]["name"]["full"]

ko_time = match_data["startTimeInUKHHMM"]

print(stadium, ko_time, formation)

In [ ]:
if data.match_list:
    print("Yes, data is there")
else:
    print("No, data is not there")

In [ ]:
trfc_players = data.tranmere_players

for player in trfc_players:
    pos = player["meta"]["positionRegular"]
    print(pos)